In [2]:
search_for = 135
start_from = 0
threads = 10
thread = 5
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.011917352676391602
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 175
all cases: 723027151
type: [1, 1, 1, 1, 135] 135
cases of this type: 332150625
100000 36.308955534476496
200000 58.8209269874375
300000 61.80843815908797
400000 68.01996442883707
500000 69.3045563190332
600000 68.71104377453312
700000 71.72335981765634
800000 69.19345811924273
900000 71.49171873277042
1000000 77.9192891851341
1100000 72.30896025984063
1200000 62.12079894060922
1300000 28.348774272305313
1400000 71.64755164449454
1500000 71.83796103837248
1600000 72.70690626720024
1700000 70.51267806635988
1800000 74.4723679433849
1900000 77.32925709537574
2000000 69.691865772051
2100000 69.79668385807507
2200000 72.22211294008493
2300000 70.43949889506688
2400000 71.87267887997771
2500000 22.62684188264523
2600000 71.12679299000139
2700000 93.46258435597163
2800000 91.42131642045993
2900000 88.76811496413507
3000000 90.72022540989329
3100

30100000 76.15368538478626
30200000 73.78001222804737
30300000 75.26093121732806
30400000 75.13825328663768
30500000 71.67362754931075
30600000 77.63718780975736
30700000 70.86313502641694
30800000 15.182851878331688
30900000 74.45838857883496
31000000 69.9645580108584
31100000 72.26449198802241
31200000 72.47252612563092
31300000 77.10904275269144
31400000 75.20054930910128
31500000 77.88156794596006
31600000 75.7452540711618
31700000 74.16018975627892
31800000 27.379719258257783
31900000 77.06725536628024
32000000 73.93234978976994
32100000 76.99450997356094
32200000 68.16319200071479
32300000 33.101789986003354
32400000 77.59479227125489
32500000 75.18270334898403
32600000 73.95066149909776
32700000 75.34305281273794
32800000 75.25582490964754
32900000 70.52463824299751
33000000 72.3521459259728
33100000 67.69170058180441
33200000 53.431575797789165
33300000 36.249397092745944
33400000 70.33881853162418
33500000 72.48916666114172
33600000 75.02832933218342
33700000 72.38622098333354

60600000 65.21345616632732
60700000 66.44061514959995
60800000 66.92627343314228
60900000 67.17685757253724
61000000 67.3915189170102
61100000 28.586015526591638
61200000 74.08101746832891
61300000 71.51388922716022
61400000 75.84618886115503
61500000 74.74453846672694
61600000 70.64675510044728
61700000 71.40506613829132
61800000 70.18746879859683
61900000 73.07925172121206
62000000 35.05567299173768
62100000 58.92697612010263
62200000 64.87198757924398
62300000 65.67986202383877
62400000 63.473212338399215
62500000 62.69466941021229
62600000 61.792994382056605
62700000 59.39371318473925
62800000 19.176923969734386
62900000 61.92475395352453
63000000 59.57687729732921
63100000 61.71328774754894
63200000 65.00168489043838
63300000 65.46712741010184
63400000 64.912139375179
63500000 51.171125211960806
63600000 48.54424329398718
63700000 71.78461896626311
63800000 69.81724954279953
63900000 67.46029532598038
64000000 70.54410806405969
64100000 70.80870360237016
64200000 72.19110074554828

90900000 64.2616702013273
91000000 62.324010140534725
91100000 67.85763407026717
91200000 64.95489184956197
91300000 67.2584736225841
91400000 66.91323062667738
91500000 63.87846070806842
91600000 65.67692075660574
91700000 50.034309809903476
91800000 34.034301978923885
91900000 60.318625826442094
92000000 61.51347893083404
92100000 62.49446665730576
92200000 58.30978682843519
92300000 19.175872748725144
92400000 57.91798175927625
92500000 59.457825821228305
92600000 56.86651284965399
92700000 59.70057857885815
92800000 45.17558545915043
92900000 40.917768984520386
93000000 63.25863819450358
93100000 64.04939821071252
93200000 65.401016585118
93300000 65.97065869127113
93400000 62.795587694400474
93500000 67.62659998482387
93600000 67.61365822397347
93700000 63.71103421617549
93800000 68.24960757536039
93900000 66.74376483873571
94000000 62.76010278795435
94100000 65.48094694740479
94200000 37.897686702688446
94300000 42.76152134363797
94400000 55.89229840983497
94500000 58.90084110746

120400000 65.55983905599858
120500000 61.119266342695084
120600000 66.7183962398261
120700000 69.1178881557208
120800000 63.392157664334185
120900000 65.2116384112466
121000000 66.83051637772026
121100000 62.03921796138379
121200000 59.610477887234644
121300000 57.479970853947385
121400000 60.783957371310116
121500000 22.854164418340403
121600000 59.0433933289887
121700000 57.41556243280944
121800000 32.35588441562113
121900000 46.66340188126336
122000000 58.67155506208773
122100000 47.235593460871584
122200000 41.2349603337583
122300000 56.34373080996146
122400000 60.23684353702727
122500000 58.7024748682064
122600000 60.94837963746773
122700000 68.50253898142115
122800000 62.08415416629384
122900000 66.78937868009065
123000000 70.15323307393646
123100000 66.33094194058393
123200000 67.73233897441612
123300000 63.64869060427166
123400000 62.89544991296815
123500000 61.61156817296043
123600000 60.1464942059215
123700000 57.55793907775096
123800000 60.658097790320234
123900000 58.394586

...

639400000 2.487228908520665
639500000 2.1826153553947796
639600000 1.9345962619880444
best so far: 0
type: [1, 1, 3, 5, 9] 135
cases of this type: 7381125
639700000 1.989342528620994
639800000 1.7531782296784417
639900000 1.7855623965956462
640000000 1.7547302603341461
640100000 1.818746681025293
640200000 1.8811846976641053
640300000 1.825347171236829
640400000 1.7554028319682842
640500000 1.7336395596363519
640600000 1.7396564144841549
640700000 1.686451554025342
640800000 1.7026995589387588
640900000 1.796472577427733
641000000 1.97612652725775
641100000 1.8526620348693172
641200000 1.6948213037148736
641300000 1.6116150910662175
641400000 1.6532311789332703
641500000 1.5767307177255492
641600000 1.6403034464785913
641700000 1.7101841815074623
641800000 1.8188001785197376
641900000 1.859863904264627
642000000 1.7946344596481
642100000 1.625404167846014
642200000 1.5763614914971584
642300000 1.4830903953857237
642400000 1.4999754290400762
642500000 1.590225990135291
642600000 1.69651

666800000 2.3015339281631753
666900000 2.466233430986273
667000000 2.4640511561212657
667100000 2.2599572104813093
667200000 2.6503989974841375
667300000 2.504344771577698
667400000 2.7121991181442655
667500000 2.703441854959114
667600000 2.607954048861911
667700000 2.7839687718476553
best so far: 0
type: [1, 1, 5, 3, 9] 135
cases of this type: 4428675
667800000 1.8229310295158796
667900000 1.0975822938025415
668000000 1.0779143033897247
668100000 1.0978249125761772
668200000 1.1110776107101799
668300000 1.113674866355622
668400000 1.0717947958559044
668500000 1.028513535475292
668600000 1.0562019079044216
668700000 1.1231370066201813
668800000 1.1372340101790188
668900000 1.0443949770268455
669000000 0.9974102406415647
669100000 0.9697998216492275
669200000 1.060868285443971
669300000 1.1795957916482587
669400000 1.170865769695876
669500000 1.0249518588588014
669600000 1.013051800775167
669700000 1.0578140690169933
669800000 1.355290035953958
669900000 1.3908004373308571
670000000 1.1

691400000 1.516294509844131
691500000 1.1742760126636962
691600000 2.0838289554158727
691700000 1.159232881580633
691800000 1.6602641561484177
691900000 1.9400371873432518
692000000 1.3235757446067027
692100000 1.7622798785563307
692200000 1.6750755093368768
692300000 1.1922181540658865
692400000 2.148554170519172
692500000 1.4647039633820989
692600000 1.3036836924739628
692700000 2.092592632998141
692800000 1.2090776802113055
692900000 1.6133381760875103
693000000 1.8280072164738985
693100000 1.4620450215734124
693200000 1.8127293737799135
693300000 1.6639230006384274
693400000 1.3206041538626392
693500000 1.873594724670418
693600000 1.3009591689301767
693700000 1.33976518881436
693800000 1.706171067141768
693900000 1.4058311690770178
694000000 1.5173335329434572
694100000 1.6021767972647012
694200000 1.4002915165517114
694300000 1.5411610733856618
694400000 1.5226692753427837
694500000 1.4401450892718302
694600000 1.272133656085483
694700000 1.5290406890732908
694800000 1.73277128971

best so far: 0
type: [1, 9, 1, 3, 5] 135
cases of this type: 151875
713000000 0.1578243206985772
713100000 0.07389766187168227
best so far: 0
type: [1, 9, 1, 5, 3] 135
cases of this type: 91125
713200000 0.08196016839334964
best so far: 0
type: [1, 9, 1, 15, 1] 135
cases of this type: 30375
best so far: 0
type: [1, 9, 3, 1, 5] 135
cases of this type: 50625
best so far: 0
type: [1, 9, 3, 5, 1] 135
cases of this type: 10125
713300000 0.07591177233981689
best so far: 0
type: [1, 9, 5, 1, 3] 135
cases of this type: 18225
best so far: 0
type: [1, 9, 5, 3, 1] 135
cases of this type: 6075
best so far: 0
type: [1, 9, 15, 1, 1] 135
cases of this type: 2025
best so far: 0
type: [1, 15, 1, 1, 9] 135
cases of this type: 98415
713400000 0.06862876164193153
best so far: 0
type: [1, 15, 1, 3, 3] 135
cases of this type: 32805
best so far: 0
type: [1, 15, 1, 9, 1] 135
cases of this type: 10935
best so far: 0
type: [1, 15, 3, 1, 3] 135
cases of this type: 10935
best so far: 0
type: [1, 15, 3, 3, 1] 135


best so far: 0
type: [9, 1, 1, 3, 5] 135
cases of this type: 16875
best so far: 0
type: [9, 1, 1, 5, 3] 135
cases of this type: 10125
best so far: 0
type: [9, 1, 1, 15, 1] 135
cases of this type: 3375
723000000 0.00025480563482575944
best so far: 0
type: [9, 1, 3, 1, 5] 135
cases of this type: 5625
best so far: 0
type: [9, 1, 3, 5, 1] 135
cases of this type: 1125
best so far: 0
type: [9, 1, 5, 1, 3] 135
cases of this type: 2025
best so far: 0
type: [9, 1, 5, 3, 1] 135
cases of this type: 675
best so far: 0
type: [9, 1, 15, 1, 1] 135
cases of this type: 225
best so far: 0
type: [9, 3, 1, 1, 5] 135
cases of this type: 1875
best so far: 0
type: [9, 3, 1, 5, 1] 135
cases of this type: 375
best so far: 0
type: [9, 3, 5, 1, 1] 135
cases of this type: 75
best so far: 0
type: [9, 5, 1, 1, 3] 135
cases of this type: 405
best so far: 0
type: [9, 5, 1, 3, 1] 135
cases of this type: 135
best so far: 0
type: [9, 5, 3, 1, 1] 135
cases of this type: 45
best so far: 0
type: [9, 15, 1, 1, 1] 135
cases 